In [1]:
"""dimension annotation
b: batch
t: token position
d: d_model

t64 means it's still a t dimension with size of 64
"""

from functools import partial

import numpy as np
import torch

import transformer_lens.utils as utils
from transformer_lens import HookedTransformer
from tqdm import tqdm

from openwebtext import load_owt, sample
from pretrained_sae import load_sae

torch.set_grad_enabled(False)

layer_index = 6
location = "resid_post_mlp"
device = utils.get_device()

ds = load_owt()
gpt2 = HookedTransformer.from_pretrained("gpt2", center_writing_weights=False)

sae_32k = load_sae(32, location, layer_index, device)

Loading dataset from disk:   0%|          | 0/152 [00:00<?, ?it/s]

Loaded 8,013,769 sample texts from data/owt_tokenized


/home/lukaemon/miniconda3/envs/topk_sae/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded pretrained model gpt2 into HookedTransformer
Loaded pretrained SAE data/sae/v5_32k_location_resid_post_mlp_layer_6.pt


In [2]:
text = 'hello world'
logit, cache = gpt2.run_with_cache(text)

In [3]:
act = cache[utils.get_act_name('resid_post', layer_index)]
act.shape

torch.Size([1, 3, 768])

In [4]:
latent, info = sae_32k.encode(act)
latent.shape

torch.Size([1, 3, 32768])

In [5]:
ac = latent[:, -1].squeeze()
(ac > 0).sum()

tensor(32, device='cuda:0')

In [6]:
torch.sort(ac[ac >0])

torch.return_types.sort(
values=tensor([ 0.6257,  0.6353,  0.6438,  0.6447,  0.6533,  0.6673,  0.6822,  0.6850,
         0.6969,  0.7092,  0.7192,  0.9029,  1.0088,  1.3280,  1.3359,  1.4029,
         1.4076,  1.5862,  1.7696,  1.7775,  1.7857,  1.7971,  2.0181,  2.2895,
         2.3179,  2.4288,  3.0338,  3.2050,  3.5099,  3.5882,  4.3947, 10.1609],
       device='cuda:0'),
indices=tensor([ 1,  8,  3,  4,  6, 14, 19, 26,  2, 17, 21, 18, 11,  0, 25, 27,  9, 28,
        12, 15,  7, 10, 22, 30, 31, 23,  5, 16, 24, 29, 20, 13],
       device='cuda:0'))